In [32]:
%cd ../content

/content


In [33]:
!apt install subversion
!svn export https://github.com/phutuan123vn/project2/trunk/Data

Reading package lists... Done
Building dependency tree       
Reading state information... Done
subversion is already the newest version (1.13.0-3ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
svn: E155000: Destination directory exists; please remove the directory or use --force to overwrite
svn: E155000: 'Data' already exists


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import copy

## **GRAPH**

In [35]:
# Copyright (c) OpenMMLab. All rights reserved.

def get_hop_distance(num_node, edge, max_hop=1):
    adj_mat = np.zeros((num_node, num_node))
    for i, j in edge:
        adj_mat[i, j] = 1
        adj_mat[j, i] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [
        np.linalg.matrix_power(adj_mat, d) for d in range(max_hop + 1)
    ]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(adj_matrix):
    Dl = np.sum(adj_matrix, 0)
    num_nodes = adj_matrix.shape[0]
    Dn = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    norm_matrix = np.dot(adj_matrix, Dn)
    return norm_matrix


def edge2mat(link, num_node):
    A = np.zeros((num_node, num_node))
    for i, j in link:
        A[j, i] = 1
    return A


class Graph:
    """The Graph to model the skeletons extracted by the openpose.

    Args:
        layout (str): must be one of the following candidates
        - openpose: 18 or 25 joints. For more information, please refer to:
            https://github.com/CMU-Perceptual-Computing-Lab/openpose#output
        - ntu-rgb+d: Is consists of 25 joints. For more information, please
            refer to https://github.com/shahroudy/NTURGB-D

        strategy (str): must be one of the follow candidates
        - uniform: Uniform Labeling
        - distance: Distance Partitioning
        - spatial: Spatial Configuration
        For more information, please refer to the section 'Partition
        Strategies' in our paper (https://arxiv.org/abs/1801.07455).

        max_hop (int): the maximal distance between two connected nodes.
            Default: 1
        dilation (int): controls the spacing between the kernel points.
            Default: 1
    """

    def __init__(self,
                 layout='coco',
                 strategy='spatial',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        assert layout in [
            'openpose-18', 'openpose-25', 'ntu-rgb+d', 'ntu_edge', 'coco'
        ]
        assert strategy in ['uniform', 'distance', 'spatial', 'agcn']
        self.get_edge(layout)
        self.hop_dis = get_hop_distance(
            self.num_node, self.edge, max_hop=max_hop)
        self.get_adjacency(strategy)

    def __str__(self):
        return self.A

    def get_edge(self, layout):
        """This method returns the edge pairs of the layout."""

        if layout == 'openpose-18':
            self.num_node = 18
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(4, 3), (3, 2), (7, 6), (6, 5),
                             (13, 12), (12, 11), (10, 9), (9, 8), (11, 5),
                             (8, 2), (5, 1), (2, 1), (0, 1), (15, 0), (14, 0),
                             (17, 15), (16, 14)]
            self.edge = self_link + neighbor_link
            self.center = 1
        elif layout == 'openpose-25':
            self.num_node = 25
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(4, 3), (3, 2), (7, 6), (6, 5), (23, 22),
                             (22, 11), (24, 11), (11, 10), (10, 9), (9, 8),
                             (20, 19), (19, 14), (21, 14), (14, 13), (13, 12),
                             (12, 8), (8, 1), (5, 1), (2, 1), (0, 1), (15, 0),
                             (16, 0), (17, 15), (18, 16)]
            self.self_link = self_link
            self.neighbor_link = neighbor_link
            self.edge = self_link + neighbor_link
            self.center = 1
        elif layout == 'ntu-rgb+d':
            self.num_node = 25
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_1base = [(1, 2), (2, 21), (3, 21),
                              (4, 3), (5, 21), (6, 5), (7, 6), (8, 7), (9, 21),
                              (10, 9), (11, 10), (12, 11), (13, 1), (14, 13),
                              (15, 14), (16, 15), (17, 1), (18, 17), (19, 18),
                              (20, 19), (22, 23), (23, 8), (24, 25), (25, 12)]
            neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
            self.self_link = self_link
            self.neighbor_link = neighbor_link
            self.edge = self_link + neighbor_link
            self.center = 21 - 1
        elif layout == 'ntu_edge':
            self.num_node = 24
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_1base = [(1, 2), (3, 2), (4, 3), (5, 2), (6, 5), (7, 6),
                              (8, 7), (9, 2), (10, 9), (11, 10), (12, 11),
                              (13, 1), (14, 13), (15, 14), (16, 15), (17, 1),
                              (18, 17), (19, 18), (20, 19), (21, 22), (22, 8),
                              (23, 24), (24, 12)]
            neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
            self.edge = self_link + neighbor_link
            self.center = 2
        elif layout == 'coco':
            self.num_node = 17
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_1base = [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13],
                              [6, 12], [7, 13], [6, 7], [8, 6], [9, 7],
                              [10, 8], [11, 9], [2, 3], [2, 1], [3, 1], [4, 2],
                              [5, 3], [4, 6], [5, 7]]
            neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
            self.edge = self_link + neighbor_link
            self.center = 0
        else:
            raise ValueError(f'{layout} is not supported.')

    def get_adjacency(self, strategy):
        """This method returns the adjacency matrix according to strategy."""

        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[
                                    i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[
                                    i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
        elif strategy == 'agcn':
            A = []
            link_mat = edge2mat(self.self_link, self.num_node)
            In = normalize_digraph(edge2mat(self.neighbor_link, self.num_node))
            outward = [(j, i) for (i, j) in self.neighbor_link]
            Out = normalize_digraph(edge2mat(outward, self.num_node))
            A = np.stack((link_mat, In, Out))
            self.A = A
        else:
            raise ValueError('Do Not Exist This Strategy')
        

if __name__ == '__main__': 
    g = Graph()

## **MODEL ST-GCN**

In [36]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable


# # from utils.tgcn import ConvTemporalGraphical
# # from utils.graph import Graph
# # import os
# # os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# class unit_gcn(nn.Module):

#     def __init__(self,
#                  in_channels,
#                  out_channels,
#                  A,
#                  adaptive='importance',
#                  conv_pos='pre',
#                  with_res=True,
#                  norm='BN',
#                  act='ReLU'):
#         super().__init__()
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.num_subsets = A.size(0)

#         assert adaptive in [None, 'init', 'offset', 'importance']
#         self.adaptive = adaptive
#         assert conv_pos in ['pre', 'post']
#         self.conv_pos = conv_pos
#         self.with_res = with_res

#         self.norm_cfg = norm if isinstance(norm, dict) else dict(type=norm)
#         self.act_cfg = act if isinstance(act, dict) else dict(type=act)
#         self.bn = nn.BatchNorm2d(out_channels)
#         self.act = nn.ReLU()

#         if self.adaptive == 'init':
#             self.A = nn.Parameter(A.clone())
#         else:
#             self.register_buffer('A', A)

#         if self.adaptive in ['offset', 'importance']:
#             self.PA = nn.Parameter(A.clone())
#             if self.adaptive == 'offset':
#                 nn.init.uniform_(self.PA, -1e-6, 1e-6)
#             elif self.adaptive == 'importance':
#                 nn.init.constant_(self.PA, 1)

#         if self.conv_pos == 'pre':
#             self.conv = nn.Conv2d(in_channels, out_channels * A.size(0), 1)
#         elif self.conv_pos == 'post':
#             self.conv = nn.Conv2d(A.size(0) * in_channels, out_channels, 1)

#         if self.with_res:
#             if in_channels != out_channels:
#                 self.down = nn.Sequential(
#                     nn.Conv2d(in_channels, out_channels, 1),
#                     nn.BatchNorm2d(out_channels))
#             else:
#                 self.down = lambda x: x

#     def forward(self, x, A=None):
#         """Defines the computation performed at every call."""
#         n, c, t, v = x.shape
#         res = self.down(x) if self.with_res else 0

#         A_switch = {None: self.A, 'init': self.A}
#         if hasattr(self, 'PA'):
#             A_switch.update({'offset': self.A + self.PA, 'importance': self.A * self.PA})
#         A = A_switch[self.adaptive]

#         if self.conv_pos == 'pre':
#             x = self.conv(x)
#             x = x.view(n, self.num_subsets, -1, t, v)
#             x = torch.einsum('nkctv,kvw->nctw', (x, A)).contiguous()
#         elif self.conv_pos == 'post':
#             x = torch.einsum('nctv,kvw->nkctw', (x, A)).contiguous()
#             x = x.view(n, -1, t, v)
#             x = self.conv(x)

#         return self.act(self.bn(x) + res)


# class unit_tcn(nn.Module):


#     def __init__(self, in_channels, out_channels, kernel_size=9, stride=1, dilation=1, norm='BN', dropout=0.5):

#         super().__init__()

#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.norm_cfg = norm if isinstance(norm, dict) else dict(type=norm)
#         pad = (kernel_size + (kernel_size - 1) * (dilation - 1) - 1) // 2

#         self.conv = nn.Conv2d(
#             in_channels,
#             out_channels,
#             kernel_size=(kernel_size, 1),
#             padding=(pad, 0),
#             stride=(stride, 1),
#             dilation=(dilation, 1))
#         self.bn = nn.BatchNorm2d(out_channels) if norm is not None else nn.Identity()
#         self.drop = nn.Dropout(dropout, inplace=True)
#         self.stride = stride

#     def forward(self, x):
#         return self.drop(self.bn(self.conv(x)))

# class STGCNBlock(nn.Module):

#     def __init__(self,
#                  in_channels,
#                  out_channels,
#                  A,
#                  stride=1,
#                  dilation=1,
#                  residual=True):
#         super().__init__()
#         self.gcn = unit_gcn(in_channels, out_channels, A)
#         self.tcn = unit_tcn(out_channels, out_channels, 9, stride=stride,dilation=dilation)
#         self.relu = nn.ReLU()
#         if not residual:
#             self.residual = lambda x: 0
#         elif (in_channels == out_channels) and (stride == 1):
#             self.residual = lambda x: x
#         else:
#             self.residual = unit_tcn(in_channels, out_channels, kernel_size=1, stride=stride)

#     def forward(self, x, A=None):
#         """Defines the computation performed at every call."""
#         res = self.residual(x)
#         x = self.tcn(self.gcn(x)) + res
#         return self.relu(x)
# class ModelSTGCN(nn.Module):
#     r"""Spatial temporal graph convolutional networks.
#     Args:
#         in_channels (int): Number of channels in the input data
#         num_class (int): Number of classes for the classification task
#         graph_args (dict): The arguments for building the graph
#         edge_importance_weighting (bool): If ``True``, adds a learnable
#             importance weighting to the edges of the graph
#         **kwargs (optional): Other parameters for graph convolution units
#     Shape:
#         - Input: :math:`(N, in_channels, T_{in}, V_{in}, M_{in})`
#         - Output: :math:`(N, num_class)` where
#             :math:`N` is a batch size,
#             :math:`T_{in}` is a length of input sequence,
#             :math:`V_{in}` is the number of graph nodes,
#             :math:`M_{in}` is the number of instance in a frame.
#     """

#     def __init__(self, in_channels, num_class,
#                  edge_importance_weighting=True):
#         super().__init__()

#         # load graph
#         self.graph = Graph()
#         A = torch.tensor(self.graph.A, dtype=torch.float32, requires_grad=False)
#         self.register_buffer('A', A)

#         # build networks
#         spatial_kernel_size = A.size(0)
#         temporal_kernel_size = 9
#         kernel_size = (temporal_kernel_size, spatial_kernel_size)
#         self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
#         self.st_gcn_networks = nn.ModuleList((
#             STGCNBlock(in_channels, 32,A=A ),
#             STGCNBlock(32, 32, A=A),
#             STGCNBlock(32, 32, A=A),
#             STGCNBlock(32, 64, A=A),
#             STGCNBlock(64, 64, A=A),
#             STGCNBlock(64, 64, A=A),
#             STGCNBlock(64, 64, A=A),
#             STGCNBlock(64, 128, A=A),
#             STGCNBlock(128, 128, A=A),
#             STGCNBlock(128, 128, A=A),
#         ))

#         # initialize parameters for edge importance weighting
#         # if edge_importance_weighting:
#         #     self.edge_importance = nn.ParameterList([
#         #         nn.Parameter(torch.ones(self.A.size()))
#         #         for i in self.st_gcn_networks
#         #     ])
#         # else:
#         #     self.edge_importance = [1] * len(self.st_gcn_networks)

#         # fcn for prediction
#         self.fcn = nn.Conv2d(128, num_class, kernel_size=1)

#     def forward(self, x):

#         # data normalization
#         # have instance of person
#         # N, C, T, V, M = x.size()
#         # x = x.permute(0, 4, 3, 1, 2).contiguous() # V C T
#         # x = x.view(N * M, V * C, T)
#         # x = self.data_bn(x)
#         # x = x.view(N, M, V, C, T)
#         # x = x.permute(0, 1, 3, 4, 2).contiguous()
#         # x = x.view(N * M, C, T, V)

#         # dont have instance of person

#         N,T,V,C = x.size()
#         x = x.permute(0, 2, 3, 1).contiguous() # N V C T
#         x = x.view(N , V * C, T)
#         x = self.data_bn(x)
#         x = x.view(N, V, C, T)
#         x = x.permute(0, 2, 3, 1).contiguous()
#         # x = x.view(N , C, T, V)
#         # forwad
#         for gcn in (self.st_gcn_networks):
#             x = gcn(x)

#         # global pooling
#         x = F.avg_pool2d(x, x.size()[2:])
#         # # x = x.view(N, M, -1, 1, 1).mean(dim=1)
#         # x = x.view(N, 1, -1, 1, 1).mean(dim=1)

#         # prediction
#         x = self.fcn(x)
#         x = x.view(x.size(0), -1)

#         return x


# # if __name__ == '__main__':
# #     print(__package__)

In [37]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable
# from utils.graph import Graph

# from utils.tgcn import ConvTemporalGraphical
# from utils.graph import Graph
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
class unit_gcn(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 A,
                 adaptive='importance',
                 conv_pos='pre',
                 with_res=True,
                 norm='BN',
                 act='ReLU'):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_subsets = A.size(0)

        assert adaptive in [None, 'init', 'offset', 'importance']
        self.adaptive = adaptive
        assert conv_pos in ['pre', 'post']
        self.conv_pos = conv_pos
        self.with_res = with_res

        self.norm_cfg = norm if isinstance(norm, dict) else dict(type=norm)
        self.act_cfg = act if isinstance(act, dict) else dict(type=act)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU()

        if self.adaptive == 'init':
            self.A = nn.Parameter(A.clone())
        else:
            self.register_buffer('A', A)

        if self.adaptive in ['offset', 'importance']:
            self.PA = nn.Parameter(A.clone())
            if self.adaptive == 'offset':
                nn.init.uniform_(self.PA, -1e-6, 1e-6)
            elif self.adaptive == 'importance':
                nn.init.constant_(self.PA, 1)

        if self.conv_pos == 'pre':
            self.conv = nn.Conv2d(in_channels, out_channels * A.size(0), 1)
        elif self.conv_pos == 'post':
            self.conv = nn.Conv2d(A.size(0) * in_channels, out_channels, 1)

        if self.with_res:
            if in_channels != out_channels:
                self.down = nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, 1),
                    nn.BatchNorm2d(out_channels))
            else:
                self.down = lambda x: x

    def forward(self, x, A=None):
        """Defines the computation performed at every call."""
        n, c, t, v = x.shape
        res = self.down(x) if self.with_res else 0

        A_switch = {None: self.A, 'init': self.A}
        if hasattr(self, 'PA'):
            A_switch.update({'offset': self.A + self.PA, 'importance': self.A * self.PA})
        A = A_switch[self.adaptive]

        if self.conv_pos == 'pre':
            x = self.conv(x)
            x = x.view(n, self.num_subsets, -1, t, v)
            x = torch.einsum('nkctv,kvw->nctw', (x, A)).contiguous()
        elif self.conv_pos == 'post':
            x = torch.einsum('nctv,kvw->nkctw', (x, A)).contiguous()
            x = x.view(n, -1, t, v)
            x = self.conv(x)

        return self.act(self.bn(x) + res)


class unit_tcn(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size=9, stride=1, dilation=1, norm='BN', dropout=0):

        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.norm_cfg = norm if isinstance(norm, dict) else dict(type=norm)
        pad = (kernel_size + (kernel_size - 1) * (dilation - 1) - 1) // 2

        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=(kernel_size, 1),
            padding=(pad, 0),
            stride=(stride, 1),
            dilation=(dilation, 1))
        self.bn = nn.BatchNorm2d(out_channels) if norm is not None else nn.Identity()
        self.drop = nn.Dropout(dropout, inplace=True)
        self.stride = stride

    def forward(self, x):
        return self.drop(self.bn(self.conv(x)))

class STGCNBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 A,
                 stride=1,
                 dilation=1,
                 residual=True):
        super().__init__()
        self.gcn = unit_gcn(in_channels, out_channels, A)
        self.tcn = unit_tcn(out_channels, out_channels, 9, stride=stride,dilation=dilation)
        self.relu = nn.ReLU()
        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = unit_tcn(in_channels, out_channels, kernel_size=1, stride=stride)

    def forward(self, x, A=None):
        """Defines the computation performed at every call."""
        res = self.residual(x)
        x = self.tcn(self.gcn(x)) + res
        return self.relu(x)
class ModelSTGCN(nn.Module):
    r"""Spatial temporal graph convolutional networks.
    Args:
        in_channels (int): Number of channels in the input data
        num_class (int): Number of classes for the classification task
        graph_args (dict): The arguments for building the graph
        edge_importance_weighting (bool): If ``True``, adds a learnable
            importance weighting to the edges of the graph
        **kwargs (optional): Other parameters for graph convolution units
    Shape:
        - Input: :math:`(N, in_channels, T_{in}, V_{in}, M_{in})`
        - Output: :math:`(N, num_class)` where
            :math:`N` is a batch size,
            :math:`T_{in}` is a length of input sequence,
            :math:`V_{in}` is the number of graph nodes,
            :math:`M_{in}` is the number of instance in a frame.
    """

    def __init__(self, in_channels, num_class,
                 edge_importance_weighting=True):
        super().__init__()

        # load graph
        self.graph = Graph()
        A = torch.tensor(self.graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)

        # build networks
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        self.st_gcn_networks = nn.ModuleList((
            STGCNBlock(in_channels, 64,A=A ),
            STGCNBlock(64, 64, A=A),
            STGCNBlock(64, 64, A=A),
            STGCNBlock(64, 64, A=A),
            STGCNBlock(64, 128, A=A),
            STGCNBlock(128, 128, A=A),
            STGCNBlock(128, 128, A=A),
            STGCNBlock(128, 256, A=A),
            STGCNBlock(256, 256, A=A),
            STGCNBlock(256, 256, A=A),
        ))

        # initialize parameters for edge importance weighting
        # if edge_importance_weighting:
        #     self.edge_importance = nn.ParameterList([
        #         nn.Parameter(torch.ones(self.A.size()))
        #         for i in self.st_gcn_networks
        #     ])
        # else:
        #     self.edge_importance = [1] * len(self.st_gcn_networks)

        # fcn for prediction
        self.fcn = nn.Conv2d(256, num_class, kernel_size=1)

    def forward(self, x):

        # data normalization
        # have instance of person
        # N, C, T, V, M = x.size()
        # x = x.permute(0, 4, 3, 1, 2).contiguous() # V C T
        # x = x.view(N * M, V * C, T)
        # x = self.data_bn(x)
        # x = x.view(N, M, V, C, T)
        # x = x.permute(0, 1, 3, 4, 2).contiguous()
        # x = x.view(N * M, C, T, V)

        # dont have instance of person

        N,T,V,C = x.size()
        x = x.permute(0, 2, 3, 1).contiguous() # N V C T
        x = x.view(N , V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        # x = x.view(N , C, T, V)
        # forwad
        for gcn in (self.st_gcn_networks):
            x = gcn(x)

        # global pooling
        x = F.avg_pool2d(x, x.size()[2:])
        # # x = x.view(N, M, -1, 1, 1).mean(dim=1)
        # x = x.view(N, 1, -1, 1, 1).mean(dim=1)

        # prediction
        x = self.fcn(x)
        x = x.view(x.size(0), -1)

        return x


# if __name__ == '__main__':
#     print(__package__)

# **Uniform Sample**


In [38]:
class UniformSampleFrames:
    """Uniformly sample frames from the video.
    To sample an n-frame clip from the video. UniformSampleFrames basically
    divide the video into n segments of equal length and randomly sample one
    frame from each segment. To make the testing results reproducible, a
    random seed is set during testing, to make the sampling results
    deterministic.
    Required keys are "total_frames", "start_index" , added or modified keys
    are "frame_inds", "clip_len", "frame_interval" and "num_clips".
    Args:
        clip_len (int): Frames of each sampled output clip.
        num_clips (int): Number of clips to be sampled. Default: 1.
        test_mode (bool): Store True when building test or validation dataset.
            Default: False.
        seed (int): The random seed used during test time. Default: 255.
    """

    def __init__(self, clip_len, num_clips=1, test_mode=False, seed=255):

        self.clip_len = clip_len
        self.num_clips = num_clips
        self.test_mode = test_mode
        self.seed = seed

    def _get_train_clips(self, num_frames, clip_len):
        """Uniformly sample indices for training clips.
        Args:
            num_frames (int): The number of frames.
            clip_len (int): The length of the clip.
        """

        assert self.num_clips == 1
        if num_frames < clip_len:
            start = np.random.randint(0, num_frames)
            inds = np.arange(start, start + clip_len)
        elif clip_len <= num_frames < 2 * clip_len:
            basic = np.arange(clip_len)
            inds = np.random.choice(
                clip_len + 1, num_frames - clip_len, replace=False)
            offset = np.zeros(clip_len + 1, dtype=np.int64)
            offset[inds] = 1
            offset = np.cumsum(offset)
            inds = basic + offset[:-1]
        else:
            bids = np.array(
                [i * num_frames // clip_len for i in range(clip_len + 1)])
            bsize = np.diff(bids)
            bst = bids[:clip_len]
            offset = np.random.randint(bsize)
            inds = bst + offset
        return inds

    def _get_test_clips(self, num_frames, clip_len):
        """Uniformly sample indices for testing clips.
        Args:
            num_frames (int): The number of frames.
            clip_len (int): The length of the clip.
        """

        np.random.seed(self.seed)
        if num_frames < clip_len:
            # Then we use a simple strategy
            if num_frames < self.num_clips:
                start_inds = list(range(self.num_clips))
            else:
                start_inds = [
                    i * num_frames // self.num_clips
                    for i in range(self.num_clips)
                ]
            inds = np.concatenate(
                [np.arange(i, i + clip_len) for i in start_inds])
        elif clip_len <= num_frames < clip_len * 2:
            all_inds = []
            for i in range(self.num_clips):
                basic = np.arange(clip_len)
                inds = np.random.choice(
                    clip_len + 1, num_frames - clip_len, replace=False)
                offset = np.zeros(clip_len + 1, dtype=np.int64)
                offset[inds] = 1
                offset = np.cumsum(offset)
                inds = basic + offset[:-1]
                all_inds.append(inds)
            inds = np.concatenate(all_inds)
        else:
            bids = np.array(
                [i * num_frames // clip_len for i in range(clip_len + 1)])
            bsize = np.diff(bids)
            bst = bids[:clip_len]
            all_inds = []
            for i in range(self.num_clips):
                offset = np.random.randint(bsize)
                all_inds.append(bst + offset)
            inds = np.concatenate(all_inds)
        return inds

    def __call__(self, results):
        num_frames = results['total_frames']

        if self.test_mode:
            inds = self._get_test_clips(num_frames, self.clip_len)
        else:
            inds = self._get_train_clips(num_frames, self.clip_len)

        inds = np.mod(inds, num_frames)
        start_index = results['start_index']
        inds = inds + start_index

        results['frame_inds'] = inds.astype(np.int64)
        results['clip_len'] = self.clip_len
        results['frame_interval'] = None
        results['num_clips'] = self.num_clips
        return results

    def __repr__(self):
        repr_str = (f'{self.__class__.__name__}('
                    f'clip_len={self.clip_len}, '
                    f'num_clips={self.num_clips}, '
                    f'test_mode={self.test_mode}, '
                    f'seed={self.seed})')
        return repr_str


In [39]:
# class UniformSampleFrames:
#     """Uniformly sample frames from the video.

#     To sample an n-frame clip from the video. UniformSampleFrames basically
#     divide the video into n segments of equal length and randomly sample one
#     frame from each segment. To make the testing results reproducible, a
#     random seed is set during testing, to make the sampling results
#     deterministic.

#     Required keys are "total_frames", "start_index" , added or modified keys
#     are "frame_inds", "clip_len", "frame_interval" and "num_clips".

#     Args:
#         clip_len (int): Frames of each sampled output clip.
#         num_clips (int): Number of clips to be sampled. Default: 1.
#         test_mode (bool): Store True when building test or validation dataset.
#             Default: False.
#         seed (int): The random seed used during test time. Default: 255.
#     """

#     def __init__(self, clip_len, num_clips=1, test_mode=False, seed=255):

#         self.clip_len = clip_len
#         self.num_clips = num_clips
#         self.test_mode = test_mode
#         self.seed = seed

#     def _get_train_clips(self, num_frames, clip_len):
#         """Uniformly sample indices for training clips.

#         Args:
#             num_frames (int): The number of frames.
#             clip_len (int): The length of the clip.
#         """

#         assert self.num_clips == 1
#         if num_frames < clip_len:
#             start = np.random.randint(0, num_frames)
#             inds = np.arange(start, start + clip_len)
#         elif clip_len <= num_frames < 2 * clip_len:
#             basic = np.arange(clip_len)
#             inds = np.random.choice(
#                 clip_len + 1, num_frames - clip_len, replace=False)
#             offset = np.zeros(clip_len + 1, dtype=np.int64)
#             offset[inds] = 1
#             offset = np.cumsum(offset)
#             inds = basic + offset[:-1]
#         else:
#             bids = np.array(
#                 [i * num_frames // clip_len for i in range(clip_len + 1)])
#             bsize = np.diff(bids)
#             bst = bids[:clip_len]
#             offset = np.random.randint(bsize)
#             inds = bst + offset
#         return inds

#     def _get_test_clips(self, num_frames, clip_len):
#         """Uniformly sample indices for testing clips.

#         Args:
#             num_frames (int): The number of frames.
#             clip_len (int): The length of the clip.
#         """

#         np.random.seed(self.seed)
#         if num_frames < clip_len:
#             # Then we use a simple strategy
#             if num_frames < self.num_clips:
#                 start_inds = list(range(self.num_clips))
#             else:
#                 start_inds = [
#                     i * num_frames // self.num_clips
#                     for i in range(self.num_clips)
#                 ]
#             inds = np.concatenate(
#                 [np.arange(i, i + clip_len) for i in start_inds])
#         elif clip_len <= num_frames < clip_len * 2:
#             all_inds = []
#             for i in range(self.num_clips):
#                 basic = np.arange(clip_len)
#                 inds = np.random.choice(
#                     clip_len + 1, num_frames - clip_len, replace=False)
#                 offset = np.zeros(clip_len + 1, dtype=np.int64)
#                 offset[inds] = 1
#                 offset = np.cumsum(offset)
#                 inds = basic + offset[:-1]
#                 all_inds.append(inds)
#             inds = np.concatenate(all_inds)
#         else:
#             bids = np.array(
#                 [i * num_frames // clip_len for i in range(clip_len + 1)])
#             bsize = np.diff(bids)
#             bst = bids[:clip_len]
#             all_inds = []
#             for i in range(self.num_clips):
#                 offset = np.random.randint(bsize)
#                 all_inds.append(bst + offset)
#             inds = np.concatenate(all_inds)
#         return inds

 

## **Some Function**

In [40]:
def Interpolation(kp, num_frames):
    """
    Repeat frames in a keypoints numpy array using linear interpolation.
    
    Args:
        kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing keypoints data.
        num_frames: integer representing the number of frames to interpolate to.
        
    Returns:
        sampled_kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing the interpolated keypoints.
    """
    # get the number of frames in the original keypoints array
    num_original_frames = kp.shape[0]
    
    # create a new array of shape (num_person, num_frames, num_keypoints, 3) to store the interpolated keypoints
    sampled_kp = np.zeros((num_frames, kp.shape[1], kp.shape[2]))
    
    # loop through each person in the keypoints array
    # loop through each keypoint in the keypoints array
    for kpt in range(kp.shape[1]):
        # loop through each coordinate in the keypoint (x, y, z)
        for coord in range(kp.shape[3]):
            # create an array of x values representing the original frames
            x = np.arange(num_original_frames)
            # create an array of x values representing the new frames
            new_x = np.linspace(0, num_original_frames-1, num_frames)
            # use linear interpolation to calculate the y values (keypoint coordinates) at the new frames
            new_y = np.interp(new_x, x, kp[:, kpt, coord])
            # store the interpolated keypoint coordinates in the new array
            sampled_kp[ :, kpt, coord] = new_y
                
    return sampled_kp

def repeat_frames(kp, num_frames):
  """
  Repeat frames in a keypoints numpy array to achieve uniform sampling.

  Args:
      kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing keypoints data.
      num_frames: integer representing the number of frames to sample to.
      
  Returns:
      sampled_kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing the keypoints after repeating frames.
  """
  # check if the number of frames in the keypoints array is less than the desired number of frames
  if kp.shape[1] < num_frames:
      # calculate the number of frames to repeat
      num_frames_to_repeat = num_frames - kp.shape[1]
      # determine the indices of the frames to repeat
      repeat_indices = np.random.choice(kp.shape[1], num_frames_to_repeat)
      # sort the indices in ascending order
      repeat_indices = np.sort(repeat_indices)
      # create an array of zeros to store the sampled keypoints
      sampled_kp = np.zeros((kp.shape[0], num_frames, kp.shape[2], kp.shape[3]))
      # loop through each person in the keypoints array
      for person in range(kp.shape[0]):
          # loop through each keypoint in the keypoints array
          for kpt in range(kp.shape[2]):
              # loop through each coordinate in the keypoint (x, y, z)
              for coord in range(kp.shape[3]):
                  # create a new array of keypoint coordinates with repeated frames
                  new_kp = np.repeat(kp[person, :, kpt, coord][np.newaxis, :], num_frames, axis=0)
                  # repeat the frames at the specified indices using linear interpolation
                  new_kp[repeat_indices] = Interpolation(kp[person, :, kpt, coord][np.newaxis, :], num_frames_to_repeat)
                  # store the new keypoint coordinates in the sampled keypoints array
                  sampled_kp[person, :, kpt, coord] = new_kp
  else:
      # if the number of frames in the keypoints array is greater than or equal to the desired number of frames, return the keypoints array as is
      sampled_kp = kp

  return sampled_kp



# **Action Dataset**

In [41]:
from copy import deepcopy
class ActionDataset(Dataset):
    def __init__(self,path, train_mode, clip_len=100):
        # super().__init__()
        # # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        # # self.transform = Transform
        # self.feature=[]
        # self.label=[]
        # # self.append(Data,label)
        self.train_mode = train_mode
        self.file = pds.read_pickle(path)
        self.clip_len = clip_len
        self.sample = UniformSampleFrames(self.clip_len)
        

    def __getitem__(self, index):
        kp = deepcopy(self.file[index]['kp'][0])
        # score = deepcopy(self.file[index]['keypoint_score'][0])
        # score = np.expand_dims(score,axis = 2)
        # kp = np.concatenate((kp,score),axis=2)
        w, h = self.file[index]['img_shape']
        kp[:,:,0] = (kp[:,:,0]-w/2)/(w/2)
        kp[:,:,1] = (kp[:,:,1]-h/2)/(h/2)
        label = deepcopy(self.file[index]['label'])
        if self.train_mode:
            inds = self.sample._get_train_clips(len(kp), self.clip_len)
        else: 
            inds = self.sample._get_test_clips(len(kp), self.clip_len)
        start_index = 5
        inds = inds + start_index
        inds = np.mod(inds, len(kp))
        kp = kp[inds] 
        # if self.transform:
            # sample=self.transform(sample)
        return torch.from_numpy(kp).float(), label
    
    def __len__(self):
        return len(self.file)
    
    # def SetTrans(self,Transform):
    #     self.transform=Transform
    
    # def append(self,Data):
    #     # super().__init__()
    #     # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
    #     # assert label==None,"Label must be give"
    #     # kpscore = np.expand_dims(Data['keypoint_score'][0],axis=2)
    #     kp = Data['kp'][0]
    #     h, w = Data['img_shape']
    #     # shapeimg=Data['img_shape']
    #     ## normalize pic
    #     # kp[:,:,0] = kp[:,:,0]/w
    #     # kp[:,:,1] = kp[:,:,1]/h
    #     kp[:,:,0] = (kp[:,:,0]-w/2)/(w/2)
    #     kp[:,:,1] = (kp[:,:,1]-h/2)/(h/2)
    #     #############
    #     # data = np.concatenate((kp,kpscore),axis=2)
    #     # data = np.expand_dims(data,axis=0)
    #     label = Data['label']
    #     kp = torch.from_numpy(kp).float()
    #     label = torch.tensor(label).long()
    #     self.feature.append(kp)
    #     self.label.append(label)
    #     self.leng=len(self.feature)  

class ToTensor():
    def __call__(self,sample):
        data,label=sample
        return torch.from_numpy(data.astype(np.float32)),torch.tensor(label)

# **Train Model**

In [42]:
def Train_epoch(model=None,loss_fn=None,train_loader=None,optimizer=None,device='cuda'):
    model.train()
    # model.training = True
    total_loss=0
    for index,(data,label) in enumerate(train_loader):
        data = data.to(device)
        outputs = model(data)
        label=label.to(device).long()
        loss = loss_fn(outputs,label)
        total_loss+=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    total_loss/=len(train_loader)
    return total_loss    

def Val_epoch(model=None,loss_fn=None,val_loader=None,device='cuda'):
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.eval()
    # model.to(device)
    total_loss = 0
    preds = []
    labels = []
    model.training = True
    with torch.no_grad():
        for index,(data,label) in enumerate(val_loader):
            data = data.to(device)
            outputs = model(data)
            label = label.to(device).long()
            loss = loss_fn(outputs,label)
            # pred = torch.argmax(outputs,dim=1)
            # labels.append(label.item())
            # preds.append(pred.item())
            total_loss+=loss
    total_loss/=len(val_loader)
    #acc multi class CrossEntropy
    # acc = eval_acc(preds,labels)
    return total_loss

def eval_acc(preds,labels):
    n_total = len(preds)
    print(n_total)
    n_correct = 0
    for pred,label in zip(preds,labels):
        if pred == label: n_correct+=1
        else: continue
    acc=n_correct/n_total
    return acc
        

## **TRAIN N EVAL**

In [52]:
def Train_n_Eval(model:nn.Module=None,epochs=None,loss_fn=None,lr=1e-4,optim:torch.optim.SGD=None,
                 train_dataloader=None,eval_dataloader=None,lr_shedule=False,
                 Step=10,miles=2,Gamma=0.1,device='cuda'):
    assert all(param is not None for param in [model,epochs,loss_fn,optim,
                                               train_dataloader,eval_dataloader]),"All Param must be give in"
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optim = optim(model.parameters(),lr,momentum=0.9,nesterov=True)
    # optim = optim(model.parameters(),lr)
    loss_history = {
        'train': [],
        'val' : [],
    }
    best_score=0
    if lr_shedule:
        Multistep=[Step * i for i in range(1,miles+1)]
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optim,Multistep,Gamma)
    for epoch in range(1,epochs+1):
        lr=optim.param_groups[-1]['lr']
        train_loss = Train_epoch(model,loss_fn,train_dataloader,optim)
        val_loss = Val_epoch(model,loss_fn,eval_dataloader)
        loss_history['train'].append(train_loss)
        loss_history['val'].append(val_loss)
        # if acc > best_score:
        #     best_score = acc
        #     model_best_wts = copy.deepcopy(model.state_dict())
        #     torch.save(model.state_dict(),'Model_best_wts.pt')
            # print("Copied best model weights!")
        if lr_shedule:
            scheduler.step()
        print(f'Epoch: {epoch}: Learning rate: {lr}\n \tTrain Loss: {train_loss}\n\tVal Loss: {val_loss}')
    model_final = copy.deepcopy(model.state_dict())
    torch.save(model.state_dict(),'model_final.pth')
    return model_final
        

## **Read File and Train**

In [48]:
train_dataset = ActionDataset('Data/train1.pkl', train_mode=True, clip_len=100)
val_dataset = ActionDataset('Data/Valid1.pkl',train_mode=False, clip_len=100)

In [53]:
model=ModelSTGCN(3,2)
criterion=nn.CrossEntropyLoss()
optim=torch.optim.SGD
# datalst1 = pds.read_pickle('Data/train1.pkl')
# datalst2 = pds.read_pickle('Data/Valid1.pkl')
train_dataset = ActionDataset('Data/train1.pkl', train_mode=True, clip_len=100)
val_dataset = ActionDataset('Data/Valid1.pkl',train_mode=False, clip_len=100)
# for data in datalst1:
#     train_dataset.append(data)
# for data in datalst2:
#     val_dataset.append(data)
train_dataloader=DataLoader(dataset=train_dataset, batch_size=30,shuffle = True)
val_dataloader=DataLoader(dataset=val_dataset, batch_size=10,shuffle = True)
model_best = Train_n_Eval(model=model,epochs=200,loss_fn=criterion,optim=optim,
                                      train_dataloader=train_dataloader,eval_dataloader=val_dataloader,
                                      lr=1e-3,lr_shedule=False,Step=15)

Epoch: 1: Learning rate: 0.001
 	Train Loss: 0.42411094903945923
	Val Loss: 0.8675338625907898
Epoch: 2: Learning rate: 0.001
 	Train Loss: 0.227898508310318
	Val Loss: 1.0990750789642334
Epoch: 3: Learning rate: 0.001
 	Train Loss: 0.2262016087770462
	Val Loss: 0.6008313298225403
Epoch: 4: Learning rate: 0.001
 	Train Loss: 0.12597350776195526
	Val Loss: 0.23241327702999115
Epoch: 5: Learning rate: 0.001
 	Train Loss: 0.1411658525466919
	Val Loss: 0.2367052137851715
Epoch: 6: Learning rate: 0.001
 	Train Loss: 0.10279134660959244
	Val Loss: 0.22950060665607452
Epoch: 7: Learning rate: 0.001
 	Train Loss: 0.09307248145341873
	Val Loss: 0.22711072862148285


KeyboardInterrupt: ignored

## **Keep Training**

In [ ]:
# model_best,model_final = Train_n_Eval(model=model,epochs=5,loss_fn=criterion,optim=optim.SGD,
#                                       train_dataloader=train_dataloader,eval_dataloader=val_dataloader,
#                                       lr=1e-3,lr_shedule=True,Step=4)

## **Eval Model**

In [54]:
torch.save(model.state_dict(),'model_modify.pth')

In [ ]:
# # model.load_state_dict(model_best)
# model.eval()
# # filetest=pds.read_pickle('/content/Data/test.pkl')
# testdata=ActionDataset('/content/Data/test.pkl',train_mode=False, clip_len=30)
# # for data in filetest:
# #     testdata.append(data)
# test_loader=DataLoader(testdata,shuffle=True,batch_size = 10)
# total_loss = 0
# preds = []
# labels = []
# with torch.no_grad():
#     for index,(data,label) in enumerate(test_loader):
#         outputs = model(data.to('cuda'))
#         label = label.to('cuda')
#         loss = criterion(outputs,label)
#         pred = torch.argmax(outputs,dim=1)
#         # labels.append(label.item())
#         # preds.append(pred.item())
#         total_loss+=loss
# total_loss/=(index + 1)
# #acc multi class CrossEntropy
# # acc = eval_acc(preds,labels)
# print(f'Total Loss: {total_loss} ')